<a href="https://colab.research.google.com/github/gpandu/GAN/blob/main/GPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tokenization:



*   Subword Tokenization : Keep frequent words and break rearer words into subwords
*   A statastical Alogrothm learns how to do this based on corpus.

> Ex: Listeria ---> "list" , "eria"

> "list" is common word likely to be found in dictionary


*   Tokenization has better chance of handling OOV words while decreasing the size of the overall dictionary.   









In [1]:
!pip install transformers
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")


In [9]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

list = tokenizer.tokenize(corpus[0])
print(list)

['This', 'Ġis', 'Ġthe', 'ĠHug', 'ging', 'ĠFace', 'ĠCourse', '.']


In [2]:
import math
import numpy as np
import tensorflow as tf

Multi Head Attention



*   Each Attention head performs Scaled Dot Product Self-Attention operation where given Keys, Query and Values, the return matrix of values given by below operation.

        Attention(Q,K,V) = softmax((Q*Transpose(K))/sqrt(d))*V





In [7]:
def scaled_dot_product_attention(query, key, value, mask=None):
  key_dims = tf.cast(tf.shape(key)[-1], tf.float32)
  scaled_scores = tf.matmul(query, key, transpose_b=True) / np.sqrt(key_dims)

  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)

  softmax = tf.keras.layers.Softmax()
  weights = softmax(scaled_scores)
  return tf.matmul(weights, value), weights



Dimensions

In [4]:
seq_len = 100
embed_dim = 100

queries = np.random.rand(seq_len, embed_dim)
keys = np.random.rand(seq_len, embed_dim)
values = np.random.rand(seq_len, embed_dim)

print("Queries:\n", queries)

Queries:
 [[0.19293793 0.46417483 0.39941747 ... 0.68786743 0.01783033 0.67943192]
 [0.73133804 0.34844797 0.7298088  ... 0.59750118 0.78490864 0.74187806]
 [0.41565966 0.79372466 0.36819985 ... 0.15490155 0.59744925 0.806301  ]
 ...
 [0.1861057  0.78924136 0.3316416  ... 0.33892491 0.02621278 0.16780694]
 [0.50536043 0.60928802 0.30687317 ... 0.86321926 0.41061945 0.57014139]
 [0.23930215 0.29813499 0.13898469 ... 0.27882105 0.47926179 0.40925589]]


In [8]:
output, attn_weights = scaled_dot_product_attention(queries, keys, values)

print("Output\n", output, "\n")
print("Weights\n", attn_weights)

Output
 tf.Tensor(
[[0.60090387 0.56057525 0.43377382 ... 0.5160571  0.5293911  0.49684963]
 [0.6038238  0.5555124  0.4321855  ... 0.5135994  0.52695465 0.4913613 ]
 [0.6058805  0.5602023  0.4331028  ... 0.51432955 0.53296757 0.49550596]
 ...
 [0.5994441  0.5590384  0.43458816 ... 0.5140016  0.5323378  0.49574932]
 [0.59967905 0.5612554  0.43213093 ... 0.51756775 0.52820635 0.49416834]
 [0.60137147 0.5590272  0.4334314  ... 0.5125146  0.5344329  0.49559245]], shape=(100, 100), dtype=float32) 

Weights
 tf.Tensor(
[[0.00884469 0.01149294 0.0118511  ... 0.01235989 0.01118493 0.00837981]
 [0.01160169 0.0136873  0.01159142 ... 0.00981132 0.01152206 0.00887288]
 [0.01038849 0.01141998 0.0136211  ... 0.01013288 0.01059768 0.00834089]
 ...
 [0.0096315  0.01177427 0.01152101 ... 0.01128797 0.01034491 0.00916195]
 [0.00883687 0.01186287 0.01200713 ... 0.01195248 0.01133723 0.00710189]
 [0.01185993 0.01245137 0.01196646 ... 0.01129662 0.01137698 0.00810285]], shape=(100, 100), dtype=float32)


**Generating queries, keys, and values for multiple heads.**

> Now that we have a way to calculate self-attention, let's actually generate the input queries, keys, and values for multiple heads.

>  each attention head had its own separate set of query, key, and value weights. Each weight matrix was of dimension  d x d/h  where h was the number of heads.




In [11]:
batch_size = 1000
seq_len = 100
embed_dim = 100
num_heads = 5
head_dim = embed_dim // num_heads

In [10]:
wq0 = np.random.rand(embed_dim, head_dim).round(1)
wq1 = np.random.rand(embed_dim, head_dim).round(1)
wq2 = np.random.rand(embed_dim, head_dim).round(1)


NameError: ignored